CSMC usage on the randomly generated matrices.

In [5]:
from tests.data_generation import create_rank_k_dataset, create_rank_k_tensor
from csmc import CSMC, NuclearNormMin
from csmc.errors.errors import approx_err
import numpy as np

n_rows = 360
n_cols = 640
rank = 10
M, M_incomplete, omega, ok_mask = create_rank_k_dataset(n_rows=n_rows, n_cols=n_cols, k=rank,
                                                        gaussian=True,
                                                        fraction_missing=0.8)

CSNN and NN methods usage. The nuclear norm minimization is solved with SCS solver.

In [6]:
solver = CSMC(M_incomplete, col_number=400)
M_filled = solver.fit_transform(M_incomplete)
approx_err(M_filled, M)

6.65356990015851e-06

In [ ]:
%%time
solver = NuclearNormMin(M_incomplete)
M_filled = solver.fit_transform(M_incomplete, missing_mask=np.isnan(M_incomplete))
approx_err(M_filled, M)

In [1]:
from tests.data_generation import create_rank_k_tensor
import torch

n_rows = 500
n_cols = 5000
rank = 5
M, M_incomplete, omega, ok_mask = create_rank_k_tensor(n_rows=n_rows, n_cols=n_cols, k=rank,
                                                        fraction_missing=0.8)


_, sigma, _ = torch.linalg.svd(M)
max_sigma = sigma[0]


lambda_grid = torch.exp(torch.linspace(torch.log(torch.tensor(0.1)),torch.log(max_sigma) - 1, 8)) 

In [ ]:
from csmc import SoftImpute, CSMC
import torch
from csmc.errors.errors import approx_err

smallest_error = None
for l in lambda_grid:
    solver = SoftImpute(M_incomplete, lambda_=l)
    M_filled = solver.fit_transform(M_incomplete, torch.isnan(M_incomplete))
    error = approx_err(M_filled, M)
    print(f"Approximation error {error} for l={l}")
    if smallest_error is None or error < smallest_error:
        smallest_error = error
        best_l = l
        

In [13]:
%%time
from csmc import SoftImpute, CSMC

from csmc.errors.errors import approx_err
solver = CSMC(M_incomplete, solver=SoftImpute, col_number=500)
M_filled = solver.fit_transform(M_incomplete)
approx_err(M_filled, M)

CPU times: user 2min 2s, sys: 227 ms, total: 2min 2s
Wall time: 15.7 s


0.05543898